In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def webScrape(links):
  unique_urls = set()
  for link in links:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    buttons = soup.find_all('a')
    url = {button['href'] for button in buttons if 'href' in button.attrs}
    unique_urls.update(url)

    for link in url:
        print(link)
    print(f"Total Links: {len(unique_urls)}")
  return unique_urls

In [3]:
url = ['https://results.eci.gov.in/']
urls = webScrape(url)

https://apps.apple.com/in/app/voter-helpline/id1456535004
https://results.eci.gov.in/AcResultGenJune2024/index.htm
https://results.eci.gov.in/PcResultGenJune2024/index.htm
https://play.google.com/store/apps/details?id=com.eci.citizen
https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
index.htm
https://results.eci.gov.in/AcResultByeJune2024/
Total Links: 7


In [5]:
filtered_urls = [url for url in urls if url.endswith('/index.htm')]
print(filtered_urls)

['https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm', 'https://results.eci.gov.in/AcResultGenJune2024/index.htm', 'https://results.eci.gov.in/PcResultGenJune2024/index.htm']


In [7]:

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve {url}")
        return None

# Function to extract and print relevant data from the parsed content
def extract_data(soup):
    # Extracting all table data
    tables = soup.find_all('table')
    for table in tables:
        if table.find('th'):  # Ensure the table has headers
            print("Extracting data from table...")
            headers = [th.text.strip() for th in table.find_all('th')]
            print(" | ".join(headers))

            rows = table.find_all('tr')[1:]  # Skip the header row
            for row in rows:
                columns = row.find_all('td')
                row_data = [column.text.strip() for column in columns]
                print(" | ".join(row_data))

# Traverse each URL
for url in filtered_urls:
    print(f"Fetching data from {url}")
    soup = fetch_data(url)
    if soup:
        extract_data(soup)
    else:
        print('Table do not exist')


Fetching data from https://results.eci.gov.in/AcResultGen2ndJune2024/index.htm
Fetching data from https://results.eci.gov.in/AcResultGenJune2024/index.htm
Fetching data from https://results.eci.gov.in/PcResultGenJune2024/index.htm
Extracting data from table...
Party | Won | Leading | Total | Total | 543 | 0 | 543
Bharatiya Janata Party - BJP | 240 | 0 | 240
Indian National Congress - INC | 99 | 0 | 99
Samajwadi Party - SP | 37 | 0 | 37
All India Trinamool Congress - AITC | 29 | 0 | 29
Dravida Munnetra Kazhagam - DMK | 22 | 0 | 22
Telugu Desam - TDP | 16 | 0 | 16
Janata Dal  (United) - JD(U) | 12 | 0 | 12
Shiv Sena (Uddhav Balasaheb Thackrey) - SHSUBT | 9 | 0 | 9
Nationalist Congress Party – Sharadchandra Pawar - NCPSP | 8 | 0 | 8
Shiv Sena - SHS | 7 | 0 | 7
Lok Janshakti Party(Ram Vilas) - LJPRV | 5 | 0 | 5
Yuvajana Sramika Rythu Congress Party - YSRCP | 4 | 0 | 4
Rashtriya Janata Dal - RJD | 4 | 0 | 4
Communist Party of India  (Marxist) - CPI(M) | 4 | 0 | 4
Indian Union Muslim League 